In [3]:
import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.utils import to_time_series_dataset
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [2]:
!pip install pandas


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: C:\Users\Usuario\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [ ]:
!pip install ipykernel

: 

In [ ]:
df = pd.read_csv("../notebooks/entregable/datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
df

In [ ]:
# 1. Preprocesar: pivotear la tabla para tener 1 fila = 1 producto
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')
df = df.sort_values(['product_id', 'periodo'])

# Asegurar consistencia temporal
pivot_df = df.pivot(index='product_id', columns='periodo', values='tn').fillna(0)

# 2. Escalar las series (opcional pero recomendado)
X = pivot_df.values
X_scaled = TimeSeriesScalerMeanVariance().fit_transform(X)

# 3. Clustering con DTW
n_clusters = 5  # Ajustar según el problema
model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", random_state=0)
labels = model.fit_predict(X_scaled)

# 4. Añadir etiquetas al DataFrame original
pivot_df['cluster'] = labels

# 5. Graficar los clústers
for cluster_id in range(n_clusters):
    plt.figure(figsize=(10, 3))
    for series in X_scaled[labels == cluster_id]:
        plt.plot(series.ravel(), alpha=0.3)
    plt.title(f'Cluster {cluster_id}')
    plt.grid(True)
    plt.show()